# 정수 인코딩 (Integer Encoding)
- 자연어 처리는 텍스트 데이터를 숫자로 변환하여 컴퓨터가 이해할 수 있도록 만드는 것이 핵심
- 정수 인코딩을 수행하여 텍스트 데이터에 고유한 인덱스를 부여 (1~5,000)
- 이러한 인코딩 과정은 전처리 과정에서 필수적이며 각 단어의 등장 빈도에 따라 인덱스를 부여하는 것이 일반적
- 단어 수를 5,000으로 제한하는 것은 모델 학습에 필요한 메모리와 계산 자원을 줄이기 위함 (등장 빈도가 낮은 단어는 제외하고 상위 5,000개 단어만 선택하는 것이 일반적)
    - -> 데이터의 노이즈를 줄이고 모델의 성능 향상 가능

In [ ]:
raw_text = """The Little Prince, written by Antoine de Saint-Exupéry, is a poetic tale about a young prince who travels from his home planet to Earth. The story begins with a pilot stranded in the Sahara Desert after his plane crashes. While trying to fix his plane, he meets a mysterious young boy, the Little Prince.

The Little Prince comes from a small asteroid called B-612, where he lives alone with a rose that he loves deeply. He recounts his journey to the pilot, describing his visits to several other planets. Each planet is inhabited by a different character, such as a king, a vain man, a drunkard, a businessman, a geographer, and a fox. Through these encounters, the Prince learns valuable lessons about love, responsibility, and the nature of adult behavior.

On Earth, the Little Prince meets various creatures, including a fox, who teaches him about relationships and the importance of taming, which means building ties with others. The fox's famous line, "You become responsible, forever, for what you have tamed," resonates with the Prince's feelings for his rose.

Ultimately, the Little Prince realizes that the essence of life is often invisible and can only be seen with the heart. After sharing his wisdom with the pilot, he prepares to return to his asteroid and his beloved rose. The story concludes with the pilot reflecting on the lessons learned from the Little Prince and the enduring impact of their friendship.

The narrative is a beautifully simple yet profound exploration of love, loss, and the importance of seeing beyond the surface of things."""

### 인코딩 처리

##### 토큰화 + 정제/정규화

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# 문장 토큰화
sentences = sent_tokenize(raw_text)

# 영문 불용어 리스트
en_stopwords = stopwords.words('english')

# 단어사전
vocab = {}

# 토큰화/정제/정규화 처리 결과
preprocessed_sentences = []

for sentence in sentences:
    sentence = sentence.lower()    # 대소문자 정규화 (소문자 변환)
    tokens = word_tokenize(sentence)    # 토큰화
    tokens = [token for token in tokens if token not in en_stopwords]    # 불용어 제거
    tokens = [token for token in tokens if len(token) > 2]    # 단어 길이가 2 이하면 제거

    for token in tokens:
        if token not in vocab:
            vocab[token] = 1
        else:
            vocab[token] += 1
    
    preprocessed_sentences.append(tokens)

In [ ]:
preprocessed_sentences

In [ ]:
vocab

##### 빈도수 기반 정제

In [ ]:
# 빈도수 기반 역순 정렬
vocab_sorted = sorted(vocab.items(), key=lambda item: item[1], reverse=True)
vocab_sorted

In [ ]:
# 인덱스 단어사전 생성
word_to_index = {word: i+1 for i, (word, cnt) in enumerate(vocab_sorted)}
word_to_index

In [ ]:
# 인덱스 단어사전2 생성
index_to_word = {i+1: word for i, (word, cnt) in enumerate(vocab_sorted)}
index_to_word

In [ ]:
vocab_size = 15
word_to_index = {word: index for word, index in word_to_index.items()\
                 if index <= vocab_size}
word_to_index

### OOV 처리

**OOV(Out of vocabulary)** : 단어사전에 정의되지 않은 단어를 가르키는 키워드

In [ ]:
word_to_index['OOV'] = len(word_to_index) + 1
word_to_index

##### 수열처리 (=정수 인코딩)

In [ ]:
encoded_sentences = []
oov_idx = word_to_index['OOV'] 

for sentence in preprocessed_sentences:
    encoded_sentence = [word_to_index.get(token, oov_idx) for token in sentence]
    print(sentence)
    print(encoded_sentence)
    print()
    encoded_sentences.append(encoded_sentence)

---

### keras Tokenizer

In [ ]:
# !pip install tensorflow

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=15, oov_token='<OOV>')

tokenizer.fit_on_texts(preprocessed_sentences)

tokenizer.word_index    # corpus의 모든 단어를 대상으로 생성

In [ ]:
tokenizer.index_word    # corpus의 모든 단어를 대상으로 생성

In [ ]:
tokenizer.word_counts    # corpus의 모든 단어를 대상으로 빈도수를 반환

In [ ]:
# 정수 인코딩
sequences = tokenizer.texts_to_sequences(preprocessed_sentences)
sequences